# Purpose:

- **set up web scraping,** 
- **create Thresold year dataset,**
- **create NESSID codes dataset,**
- **create list of 4-digit codes for every RAWS called station_list**


--------------------------------------------------------------------
# Summary 

1. **Create Menu list**
    - list of every state and sectioned by the websites region
    - Taken from web inspect
    
    
2. **Dictionary of each RAWS url**


3. **EDA**


4. **Pulling results**
    - Creating Threshold Year
        - Used to set RAWS year limit, if a station stops collecting data in 2019 every day after will be filled with nulls, this list is used to ensure this limit is enforced when loading data in EDA1, so after 2019 the rows would be dropped instead. 
        
        
5. **Pulling NESSID CODES**
    - Creating NESSID.csv 
    - webscrape each RAWS NESSID and 4-digit code 
    - Allows the final dataset be joined with other RAWS datasets
    
    
6. **Random Extra EDA**


7. **EDA Part 2**


8. **Pulling Final List of 4-digit Codes**


9. **Creating Station_list**
    - This is the chunked csv used to create post requests 


10. **Random Functions**

In [ ]:
from joblib import Parallel, delayed
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pickle
import time
import re

# 1. Create Menu list

In [4]:
menu = {
    "Western U.S.": {
        "Alaska": "akF.html",
        "Arizona": "azF.html",
        "California, Northern": "ncaF.html",
        "California, Central": "ccaF.html",
        "California, Southern": "scaF.html",
        "Colorado": "coF.html",
        "Hawaii": "hiF.html",
        "Idaho, Northern": "nidwmtF.html",
        "Idaho, Southern": "sidF.html",
        "Montana, Eastern": "emtF.html",
        "Montana, Western": "nidwmtF.html",
        "Nevada": "nvF.html",
        "New Mexico": "nmF.html",
        "Oregon": "orF.html",
        "Utah": "utF.html",
        "Washington": "waF.html",
        "Wyoming": "wyF.html"
    },
    "Central U.S.": {
        "Illinois": "ilF.html",
        "Indiana": "inF.html",
        "Iowa": "iaF.html",
        "Kansas": "ksF.html",
        "Kentucky": "ky_tnF.html",
        "Michigan": "mi_wiF.html",
        "Minnesota": "mnF.html",
        "Missouri": "moF.html",
        "Nebraska": "neF.html",
        "North Dakota": "ndF.html",
        "Ohio": "ohF.html",
        "South Dakota": "sdF.html",
        "Wisconsin": "mi_wiF.html"
    },
    "Southern U.S.": {
        "Alabama": "al_msF.html",
        "Arkansas": "arF.html",
        "Florida": "flF.html",
        "Georgia": "ga_scF.html",
        "Louisiana": "laF.html",
        "Mississippi": "al_msF.html",
        "North Carolina": "ncF.html",
        "Oklahoma": "okF.html",
        "South Carolina": "ga_scF.html",
        "Tennessee": "ky_tnF.html",
        "Texas": "txF.html",
        "Puerto Rico": "prF.html"
    },
    "Northeastern U.S.": {
        "Connecticut": "ct_ma_riF.html",
        "Delaware": "de_mdF.html",
        "Maine": "me_nh_vtF.html",
        "Maryland": "de_mdF.html",
        "Massachusetts": "ct_ma_riF.html",
        "New Hampshire": "me_nh_vtF.html",
        "New Jersey": "nj_paF.html",
        "New York": "nyF.html",
        "Pennsylvania": "nj_paF.html",
        "Rhode Island": "ct_ma_riF.html",
        "Vermont": "me_nh_vtF.html",
        "Virginia": "va_wvF.html",
        "West Virginia": "va_wvF.html"
    }
}

stationsMenu = {}

for region, states in menu.items():
    stationsMenu[region] = {}
    for state, url in states.items():
        stationsMenu[region][state] = url.replace("F.html", "lst.html")


# 2. Dictionary of each RAWS url

In [387]:

raws_urls = {} 
# Iterate over each state
for region, states in stationsMenu.items():
    print(f"{'='*100}")
    print(f"\n\033[91mRegion: {region}\033[0m\n")
    print(f"{'-'*100}")
    for state, url in states.items():
        full_url = f"https://raws.dri.edu/{url}"
        print(f"State: {state}")
        print(f"URL: {full_url}")

        # Retrieve HTML content of the website
        response = requests.get(full_url)
        html_content = response.text

        # BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the RAWS sites
        raws_links = soup.find_all('a', {'href': re.compile(r'.+')})

        # Iterate over the RAWS sites, and create url for each one
        for raws_link in raws_links:
            # Format is bs4.element.Tag so key=value, href is the key i want, i want the id so after the ?
            href_parts = raws_link['href'].split('?')
            # last one is a email, has len of 1 so this removes it
            if len(href_parts) >= 2:
                raws_url = f"https://raws.dri.edu/cgi-bin/wea_dysimts.pl?{href_parts[1]}"
                raws_name = raws_link.text.strip()

                print(f"RAWS Site: {raws_name}")
                print(f"RAWS URL: {raws_url}")

                # Adding RAWS URL to dictionary raws_urls
                raws_urls[raws_name] = raws_url

print(len(raws_urls)) 



Region: Western U.S.

----------------------------------------------------------------------------------------------------
State: Alaska
URL: https://raws.dri.edu/aklst.html
RAWS Site: 100 Mile Creek        Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akA100
RAWS Site: Alcan Hwy MI-1244       Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAALC
RAWS Site: Angel Creek  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAANG
RAWS Site: Asik  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAASI
RAWS Site: Berners Bay  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABBY
RAWS Site: Black Cape  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABCA
RAWS Site: Birch Creek  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABCK
RAWS Site: Bear Mtn.         Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABEA
RAWS Site: Ben Creek              Alaska
RAWS URL: https://raws.dri.edu/

RAWS Site: Aqua Canyon - Bryce Canyon NP.
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utAQUA
RAWS Site: Assay - Hatch 10SW
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utASSA
RAWS Site: Badger Spring - Ivins 14W  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utBADG
RAWS Site: Barney Reservoir - Escalante 15SE
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utBARN
RAWS Site: Beaver Dam  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utBEAV
RAWS Site: Bryce Canyon  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utBRYC
RAWS Site: Enterprise  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utENTR
RAWS Site: Kane Gulch - Blanding 23WSW
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utKANE
RAWS Site: Lava Point             Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utLAVA
RAWS Site: Telegraph Flat - Kanab 17E  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utTELE
RAWS Site: Buckskin  U

RAWS Site: Barrel Springs      Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBAR
RAWS Site: Bluewing Mountain   Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBLU
RAWS Site: Brawley Peaks       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBRA
RAWS Site: Black Rock Playa  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBRP
RAWS Site: Buffalo Creek       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBUF
RAWS Site: Catnip Mountain     Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCAT
RAWS Site: Cherry Valley  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCHV
RAWS Site: Coyote Canyon       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCYC
RAWS Site: Dead Camel Mountain Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDEA
RAWS Site: Double H  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDOU
RAWS Site: Dry Canyon      

RAWS Site: Ash Meadows  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNASH
RAWS Site: Austin  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNAUS
RAWS Site: Buddy Adams  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBDY
RAWS Site: Brawley Peaks       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBRA
RAWS Site: Cherry Valley  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCHV
RAWS Site: Coils Creek         Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCOI
RAWS Site: Combs Canyon        Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCOM
RAWS Site: Dead Camel Mountain Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDEA
RAWS Site: Desatoya Mountain   Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDES
RAWS Site: Desert Springs       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDSS
RAWS Site: Fish Springs        Nevada
RA

RAWS Site: Ash Meadows  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNASH
RAWS Site: Big Bend            Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBIG
RAWS Site: Christmas Tree Pass Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCHR
RAWS Site: Desert NWR  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDNW
RAWS Site: Kyle Canyon  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNKYL
RAWS Site: Mountain Springs  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNMOU
RAWS Site: Pahrump             Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNPAH
RAWS Site: Red Rock            Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNRER
RAWS Site: Toquop Wash         Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNTOQ
RAWS Site: Wamp Springs  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNWAM
RAWS Site: Yucca Gap  Nevada
RAWS URL: h

...

RAWS Site: Isle au Haut            Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMISL
RAWS Site: McFarland Hill  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMMCF
RAWS Site: Moosehorn  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMMOO
RAWS Site: Oxbow  Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMOXB
RAWS Site: Rachel Carson  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMRCA
RAWS Site: Sunkhaze Meadows  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMSUN
RAWS Site: Turner Brook  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMTUR
RAWS Site: Bear Brook  New Hampshire
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNBRB
RAWS Site: Great Gulf         New Hampshire
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNGRE
RAWS Site: Lancaster  New Hampshire
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNLAN
RAWS Site: Sara  New York
RAWS URL: https

RAWS Site: Ancora Hospital  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJANC
RAWS Site: Blue Mountain Lakes  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJBLU
RAWS Site: Coyle Field  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJCOY
RAWS Site: Jackson  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJJAC
RAWS Site: New Middlesex County  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJMID
RAWS Site: Ringwood  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJRIN
RAWS Site: Teetertown  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJTEE
RAWS Site: Wallkill River NWR  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJWAL
RAWS Site: Blue Hills  Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMBLH
RAWS Site: Cape Cod          Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMCAP
RAWS Site:

RAWS Site: Blue Mountain Lakes  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJBLU
RAWS Site: Ringwood  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJRIN
RAWS Site: Teetertown  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJTEE
RAWS Site: Wallkill River NWR  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJWAL
RAWS Site: Fire Island           New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNFIR
RAWS Site: Gang Mills  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNGAN
RAWS Site: Sara  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNSAR
RAWS Site: Schroon Lake  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNSCH
RAWS Site: Sherburne  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNSHR
RAWS Site: Stonykill  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNSTO
RAWS Site: FRWS4 Church & Cortlandt
RAWS UR

RAWS Site: Ancora Hospital  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJANC
RAWS Site: Blue Mountain Lakes  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJBLU
RAWS Site: Coyle Field  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJCOY
RAWS Site: Jackson  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJJAC
RAWS Site: New Middlesex County  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJMID
RAWS Site: Ringwood  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJRIN
RAWS Site: Teetertown  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJTEE
RAWS Site: Wallkill River NWR  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJWAL
RAWS Site: Blue Hills  Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMBLH
RAWS Site: Cape Cod          Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMCAP
RAWS Site:

RAWS Site: Blackbird  Delaware
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txDBLB
RAWS Site: Prime Hook  Delaware
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txDPRI
RAWS Site: Redden  Delaware
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txDRED
RAWS Site: Big Sandy  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txKBSN
RAWS Site: Cherry Tree Knob  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txKCHE
RAWS Site: Jackson Co. AP  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txKJAC
RAWS Site: Triangle Mtn  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txKTRI
RAWS Site: Antietam NB  Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMANN
RAWS Site: Assateague Island     Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMASS
RAWS Site: Blackwater  Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMBLA
RAWS Site: Cedarville  Maryland
RAWS URL: https://raw

In [5]:
raws_urls1 = {} 
check_urls = set()
# Same as above but the raws_links process is slightly different
# same outcome
# Iterate over each state
for region, states in stationsMenu.items():
    print(f"{'='*100}")
    print(f"\n\033[91mRegion: {region}\033[0m\n")
    print(f"{'-'*100}")
    for state, url in states.items():
        full_url = f"https://raws.dri.edu/{url}"
        print(f"State: {state}")
        print(f"URL: {full_url}")

        # Retrieve HTML content of the website
        response = requests.get(full_url)
        html_content = response.text

        # BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the RAWS sites
        raws_links = soup.find_all('a', {'href': re.compile(r'.+')})

        # Iterate over the RAWS sites, and create url for each one
        for raws_link in raws_links:
            if raws_link['href'][-4:].isupper():
                raws_url1 = f"https://raws.dri.edu/cgi-bin/wea_dysimts.pl?{raws_link['href'][-6:]}"
                raws_name1 = raws_link.text.strip()

                print(f"RAWS Site: {raws_name1}")
                print(f"RAWS URL: {raws_url1}")

                # Adding RAWS URL to dictionary raws_urls
                raws_urls1[raws_name1] = raws_url1
                check_urls.add(raws_url1)

print(len(raws_urls1)) 


Region: Western U.S.

----------------------------------------------------------------------------------------------------
State: Alaska
URL: https://raws.dri.edu/aklst.html
RAWS Site: 100 Mile Creek        Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akA100
RAWS Site: Alcan Hwy MI-1244       Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAALC
RAWS Site: Angel Creek  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAANG
RAWS Site: Asik  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAASI
RAWS Site: Berners Bay  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABBY
RAWS Site: Black Cape  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABCA
RAWS Site: Birch Creek  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABCK
RAWS Site: Bear Mtn.         Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABEA
RAWS Site: Ben Creek              Alaska
RAWS URL: https://raws.dri.edu/

RAWS Site: Aqua Canyon - Bryce Canyon NP.
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utAQUA
RAWS Site: Assay - Hatch 10SW
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utASSA
RAWS Site: Badger Spring - Ivins 14W  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utBADG
RAWS Site: Barney Reservoir - Escalante 15SE
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utBARN
RAWS Site: Beaver Dam  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utBEAV
RAWS Site: Bryce Canyon  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utBRYC
RAWS Site: Enterprise  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utENTR
RAWS Site: Kane Gulch - Blanding 23WSW
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utKANE
RAWS Site: Lava Point             Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utLAVA
RAWS Site: Telegraph Flat - Kanab 17E  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?utTELE
RAWS Site: Buckskin  U

RAWS Site: Barrel Springs      Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBAR
RAWS Site: Bluewing Mountain   Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBLU
RAWS Site: Brawley Peaks       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBRA
RAWS Site: Black Rock Playa  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBRP
RAWS Site: Buffalo Creek       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBUF
RAWS Site: Catnip Mountain     Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCAT
RAWS Site: Cherry Valley  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCHV
RAWS Site: Coyote Canyon       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCYC
RAWS Site: Dead Camel Mountain Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDEA
RAWS Site: Double H  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDOU
RAWS Site: Dry Canyon      

RAWS Site: Ash Meadows  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNASH
RAWS Site: Austin  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNAUS
RAWS Site: Buddy Adams  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBDY
RAWS Site: Brawley Peaks       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBRA
RAWS Site: Cherry Valley  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCHV
RAWS Site: Coils Creek         Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCOI
RAWS Site: Combs Canyon        Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCOM
RAWS Site: Dead Camel Mountain Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDEA
RAWS Site: Desatoya Mountain   Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDES
RAWS Site: Desert Springs       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDSS
RAWS Site: Fish Springs        Nevada
RA

RAWS Site: Ash Meadows  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNASH
RAWS Site: Big Bend            Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNBIG
RAWS Site: Christmas Tree Pass Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNCHR
RAWS Site: Desert NWR  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNDNW
RAWS Site: Kyle Canyon  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNKYL
RAWS Site: Mountain Springs  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNMOU
RAWS Site: Pahrump             Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNPAH
RAWS Site: Red Rock            Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNRER
RAWS Site: Toquop Wash         Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNTOQ
RAWS Site: Wamp Springs  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNWAM
RAWS Site: Yucca Gap  Nevada
RAWS URL: h

...

RAWS Site: Isle au Haut            Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMISL
RAWS Site: McFarland Hill  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMMCF
RAWS Site: Moosehorn  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMMOO
RAWS Site: Oxbow  Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMOXB
RAWS Site: Rachel Carson  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMRCA
RAWS Site: Sunkhaze Meadows  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMSUN
RAWS Site: Turner Brook  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMTUR
RAWS Site: Bear Brook  New Hampshire
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNBRB
RAWS Site: Great Gulf         New Hampshire
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNGRE
RAWS Site: Lancaster  New Hampshire
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNLAN
RAWS Site: Monadnock  New Hampshire
RAWS 

RAWS Site: Ancora Hospital  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJANC
RAWS Site: Blue Mountain Lakes  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJBLU
RAWS Site: Coyle Field  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJCOY
RAWS Site: Jackson  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJJAC
RAWS Site: New Middlesex County  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJMID
RAWS Site: Ringwood  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJRIN
RAWS Site: Teetertown  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJTEE
RAWS Site: Wallkill River NWR  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJWAL
RAWS Site: Blue Hills  Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMBLH
RAWS Site: Cape Cod          Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMCAP
RAWS Site:

RAWS Site: Blue Mountain Lakes  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJBLU
RAWS Site: Ringwood  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJRIN
RAWS Site: Teetertown  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJTEE
RAWS Site: Wallkill River NWR  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJWAL
RAWS Site: Fire Island           New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNFIR
RAWS Site: Gang Mills  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNGAN
RAWS Site: Sara  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNSAR
RAWS Site: Schroon Lake  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNSCH
RAWS Site: Sherburne  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNSHR
RAWS Site: Stonykill  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txNSTO
RAWS Site: FRWS4 Church & Cortlandt
RAWS UR

RAWS Site: Ancora Hospital  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJANC
RAWS Site: Blue Mountain Lakes  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJBLU
RAWS Site: Coyle Field  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJCOY
RAWS Site: Jackson  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJJAC
RAWS Site: New Middlesex County  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJMID
RAWS Site: Ringwood  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJRIN
RAWS Site: Teetertown  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJTEE
RAWS Site: Wallkill River NWR  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJWAL
RAWS Site: Blue Hills  Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMBLH
RAWS Site: Cape Cod          Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMCAP
RAWS Site:

RAWS Site: Blackbird  Delaware
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txDBLB
RAWS Site: Prime Hook  Delaware
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txDPRI
RAWS Site: Redden  Delaware
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txDRED
RAWS Site: Big Sandy  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txKBSN
RAWS Site: Cherry Tree Knob  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txKCHE
RAWS Site: Jackson Co. AP  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txKJAC
RAWS Site: Triangle Mtn  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txKTRI
RAWS Site: Antietam NB  Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMANN
RAWS Site: Assateague Island     Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMASS
RAWS Site: Blackwater  Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txMBLA
RAWS Site: Cedarville  Maryland
RAWS URL: https://raw

# 3. EDA
- Explained at the end of the section

In [17]:
check_urls

{'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCWOO',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?idICIT',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?laABNS',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?ndKSTA',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?ndTSAU',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOMET',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?coCSDV',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCCAE',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?waWBAI',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCLAP',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAFAR',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOTTM',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOMID',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?ndNSCK',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txJMID',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCPRT',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?laFLKG',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?waWMRB',
 'https://

In [23]:
len(raws_urls1)

2907

In [46]:
len(check_urls)

2868

In [47]:
len(raws_urls1) -  len(check_urls)

39

In [38]:
'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?azACHR' in check_urls

True

In [ ]:
def findDup(my_dict):
    '''
    Function used to find duplicate values in a dictionary
    my_dict : input dictionary
    '''
    
    duplicate_values_dict = {}
    seen_values = []
    duplicate_values = []
    # Add value to 
    for key, value in my_dict.items():
        # if value is in 
        if value in seen_values:
            duplicate_values.append(value)
        seen_values.append(value)

    for key, value in my_dict.items():
        if value in duplicate_values:
            duplicate_values_dict[key] = value

    sorted_dict = dict(sorted(duplicate_values_dict.items(), key=lambda x: x[1]))   
    

    list(map(print, sorted_dict.items()))
    return(sorted_dict.items(), duplicate_values)

In [51]:
ll, lll = findDup(raws_urls1)
len(ll)

('Headquarters         Arizona', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?azACHR')
('Chiricahua         Arizona', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?azACHR')
('Jack Wright  Nevada', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCJAK')
('Jack Wright  California', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCJAK')
('Red Mountain       California', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCSQS')
('Squaw Springs       California', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCSQS')
('Gaylor Meadow      California', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCTUO')
('Tuolumne Meadows      California', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCTUO')
('Lodore             Colorado', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?coCLAD')
('Ladore             Colorado', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?coCLAD')
('Moburg Canyon       Idaho', 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?idIMOL')
('Molburg Canyon       Idaho', 'https://raws.dri.edu/cgi

86

In [59]:
len(set(ll)) /2

43.0

1. some have different names but the same code and same url, this results in duplicates in the dict, but not the list.
2. Next we have 8 codes that share the same name but different code, **(4 groups of 2 share same code)**
    - the codes that get droped are ['MWHM', 'CLVQ', 'HPHU', 'SSFS']
    - same name, different code
    - only impacts dictionary, not list of codes
    
    
Totals:
- 43 extra in dict
- 4 missing in dict


**Total: 39**

# 4. Pulling results
- results created with check_url

In [6]:
# Creating check_url
check_url = set(check_urls)
len(check_url)

2892

In [8]:
def find_latest_date(url):
    final_urls = {}

    response = requests.get(url)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Convert soup to string
        html_content = str(soup)

        # Search for the pattern and extract the rest of the string
        pattern = r'Latest available data:(.*)'
        result = re.search(pattern, html_content)

        try:
            if int(result.group(1)[-5:-1]) > 2014:
#                     final_urls[url] = int(result.group(1)[-5:-1])
                return {url : int(result.group(1)[-5:-1])}
        except Exception as e:
            print(f'{e}\n{url}')
    else:
        pass
# find_latest_date(check_url)

# Process URLs in parallel
results = Parallel(n_jobs=-1, verbose=1)(
delayed(find_latest_date)(url) for url in check_urls)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 2892 out of 2892 | elapsed:  7.8min finished


In [30]:
rr = {key: value for r in results if r is not None for key, value in r.items()}
len(rr)

2259

In [33]:
modified_dict = {}

for key, value in rr.items():
    modified_key = key[-4:]
    modified_dict[modified_key] = value

modified_dict

{'MSUN': 2016,
 'HDIL': 2023,
 'XCHU': 2023,
 'SBAK': 2023,
 'AFLG': 2023,
 'SFOP': 2023,
 'CSLA': 2023,
 'CDIA': 2023,
 'OROB': 2023,
 'COWE': 2023,
 'YCAT': 2023,
 'XBAR': 2023,
 'OLAV': 2023,
 'CSP3': 2023,
 'HP17': 2023,
 'MUDS': 2023,
 'GWAS': 2023,
 'CBRF': 2023,
 'WSMI': 2023,
 'OROC': 2023,
 'HOAH': 2023,
 'OFOS': 2023,
 'ILCR': 2023,
 'MWAU': 2023,
 'IROC': 2023,
 'TFDV': 2023,
 'MLAB': 2023,
 'SBEA': 2023,
 'VFEO': 2023,
 'HKMO': 2023,
 'CWHH': 2023,
 'MCPK': 2023,
 'FLWR': 2023,
 'CLAP': 2023,
 'MWIC': 2023,
 'ADON': 2023,
 'CSHQ': 2023,
 'CBRA': 2023,
 'OWAR': 2023,
 'VASH': 2023,
 'ASDH': 2023,
 'CDCB': 2023,
 'DRED': 2023,
 'CCHI': 2023,
 'CRUT': 2023,
 'OHOY': 2023,
 'GATH': 2023,
 'ACHP': 2022,
 'TPAN': 2023,
 'WLAO': 2023,
 'XGIL': 2023,
 'IDEA': 2023,
 'WGRE': 2023,
 'ASTU': 2023,
 'CBAC': 2023,
 'NLON': 2023,
 'CBRO': 2023,
 'AINN': 2023,
 'WLES': 2023,
 'MROS': 2023,
 'CRIF': 2023,
 'CSIM': 2023,
 'WOWE': 2023,
 'ICRS': 2023,
 'MMNR': 2023,
 'TTEM': 2023,
 'CFEN': 2

In [34]:
# Saving results
# with open('threshold_year.pickle', 'wb') as file:
#     pickle.dump(modified_dict, file)

## Creating f_list
- results creates final_dict
- final_dict creates f_list
- f_list creates result (down below)

In [682]:
final_dict = {key: value for r in results if r is not None for key, value in r.items()}

final_dict  

{'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txPALL': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txVFDE': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCNEW': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCOPA': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOKE2': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOPEB': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNGAL': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?hiHKII': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?laNJES': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOCIN': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?ndITWR': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?coCLOO': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?azANAV': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCHOR': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?laLEVA': 2023,
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCSP3': 2023,
 'https://raws.dri.edu/c

In [687]:
# Used to create list chunks
f_list = [end[-4:] for end in list(final_dict.keys())]

In [689]:
len(f_list)

2252

In [691]:
len(set(f_list))

2252

**--------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# 5. Pulling NESSID CODES
- I am pulling from all stations will just drop later

In [4]:
raws_urlsc = {} 
code_urls = set()
# Same as above but pulls a different url
# Iterate over each state
for region, states in stationsMenu.items():
    print(f"{'='*100}")
    print(f"\n\033[91mRegion: {region}\033[0m\n")
    print(f"{'-'*100}")
    for state, url in states.items():
        full_url = f"https://raws.dri.edu/{url}"
        print(f"State: {state}")
        print(f"URL: {full_url}")

        # Retrieve HTML content of the website
        response = requests.get(full_url)
        html_content = response.text

        # BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the RAWS sites
        raws_links = soup.find_all('a', {'href': re.compile(r'.+')})

        # Iterate over the RAWS sites, and create url for each one
        for raws_link in raws_links:
            if raws_link['href'][-4:].isupper():
                raws_urlc = f"https://raws.dri.edu/cgi-bin/wea_info.pl?{raws_link['href'][-6:]}"
                raws_namec = raws_link.text.strip()

                print(f"RAWS Site: {raws_namec}")
                print(f"RAWS URL: {raws_urlc}")

                # Adding RAWS URL to dictionary raws_urls
                raws_urlsc[raws_namec] = raws_urlc
                code_urls.add(raws_urlc)

print(len(raws_urlsc)) 


Region: Western U.S.

----------------------------------------------------------------------------------------------------
State: Alaska
URL: https://raws.dri.edu/aklst.html
RAWS Site: 100 Mile Creek        Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?akA100
RAWS Site: Alcan Hwy MI-1244       Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?akAALC
RAWS Site: Angel Creek  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?akAANG
RAWS Site: Asik  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?akAASI
RAWS Site: Berners Bay  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?akABBY
RAWS Site: Black Cape  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?akABCA
RAWS Site: Birch Creek  Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?akABCK
RAWS Site: Bear Mtn.         Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?akABEA
RAWS Site: Ben Creek              Alaska
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?akAB

RAWS Site: Aqua Canyon - Bryce Canyon NP.
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utAQUA
RAWS Site: Assay - Hatch 10SW
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utASSA
RAWS Site: Badger Spring - Ivins 14W  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utBADG
RAWS Site: Barney Reservoir - Escalante 15SE
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utBARN
RAWS Site: Beaver Dam  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utBEAV
RAWS Site: Bryce Canyon  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utBRYC
RAWS Site: Enterprise  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utENTR
RAWS Site: Kane Gulch - Blanding 23WSW
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utKANE
RAWS Site: Lava Point             Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utLAVA
RAWS Site: Telegraph Flat - Kanab 17E  Utah
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?utTELE
RAWS Site: Buckskin  Utah
RAWS URL: https://raws.dri

RAWS Site: Barrel Springs      Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNBAR
RAWS Site: Bluewing Mountain   Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNBLU
RAWS Site: Brawley Peaks       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNBRA
RAWS Site: Black Rock Playa  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNBRP
RAWS Site: Buffalo Creek       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNBUF
RAWS Site: Catnip Mountain     Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNCAT
RAWS Site: Cherry Valley  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNCHV
RAWS Site: Coyote Canyon       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNCYC
RAWS Site: Dead Camel Mountain Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNDEA
RAWS Site: Double H  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNDOU
RAWS Site: Dry Canyon          Nevada
RAWS URL: https://r

RAWS Site: Ash Meadows  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNASH
RAWS Site: Austin  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNAUS
RAWS Site: Buddy Adams  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNBDY
RAWS Site: Brawley Peaks       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNBRA
RAWS Site: Cherry Valley  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNCHV
RAWS Site: Coils Creek         Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNCOI
RAWS Site: Combs Canyon        Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNCOM
RAWS Site: Dead Camel Mountain Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNDEA
RAWS Site: Desatoya Mountain   Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNDES
RAWS Site: Desert Springs       Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNDSS
RAWS Site: Fish Springs        Nevada
RAWS URL: https://raws.dri.edu/c

RAWS Site: Ash Meadows  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNASH
RAWS Site: Big Bend            Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNBIG
RAWS Site: Christmas Tree Pass Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNCHR
RAWS Site: Desert NWR  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNDNW
RAWS Site: Kyle Canyon  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNKYL
RAWS Site: Mountain Springs  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNMOU
RAWS Site: Pahrump             Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNPAH
RAWS Site: Red Rock            Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNRER
RAWS Site: Toquop Wash         Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNTOQ
RAWS Site: Wamp Springs  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?nvNWAM
RAWS Site: Yucca Gap  Nevada
RAWS URL: https://raws.dri.edu/cgi-bin/we

...

RAWS Site: Isle au Haut            Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMISL
RAWS Site: McFarland Hill  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMMCF
RAWS Site: Moosehorn  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMMOO
RAWS Site: Oxbow  Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMOXB
RAWS Site: Rachel Carson  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMRCA
RAWS Site: Sunkhaze Meadows  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMSUN
RAWS Site: Turner Brook  Maine
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMTUR
RAWS Site: Bear Brook  New Hampshire
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txNBRB
RAWS Site: Great Gulf         New Hampshire
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txNGRE
RAWS Site: Lancaster  New Hampshire
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txNLAN
RAWS Site: Sara  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_in

RAWS Site: Ancora Hospital  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJANC
RAWS Site: Blue Mountain Lakes  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJBLU
RAWS Site: Coyle Field  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJCOY
RAWS Site: Jackson  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJJAC
RAWS Site: New Middlesex County  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJMID
RAWS Site: Ringwood  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJRIN
RAWS Site: Teetertown  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJTEE
RAWS Site: Wallkill River NWR  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJWAL
RAWS Site: Blue Hills  Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMBLH
RAWS Site: Cape Cod          Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMCAP
RAWS Site: Camp Edwards  Massachusetts
R

RAWS Site: Blue Mountain Lakes  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJBLU
RAWS Site: Ringwood  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJRIN
RAWS Site: Teetertown  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJTEE
RAWS Site: Wallkill River NWR  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJWAL
RAWS Site: Fire Island           New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txNFIR
RAWS Site: Gang Mills  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txNGAN
RAWS Site: Sara  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txNSAR
RAWS Site: Schroon Lake  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txNSCH
RAWS Site: Sherburne  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txNSHR
RAWS Site: Stonykill  New York
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txNSTO
RAWS Site: FRWS4 Church & Cortlandt
RAWS URL: https://raws.dri.edu/cgi-bi

RAWS Site: Ancora Hospital  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJANC
RAWS Site: Blue Mountain Lakes  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJBLU
RAWS Site: Coyle Field  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJCOY
RAWS Site: Jackson  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJJAC
RAWS Site: New Middlesex County  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJMID
RAWS Site: Ringwood  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJRIN
RAWS Site: Teetertown  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJTEE
RAWS Site: Wallkill River NWR  New Jersey
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txJWAL
RAWS Site: Blue Hills  Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMBLH
RAWS Site: Cape Cod          Massachusetts
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMCAP
RAWS Site: Camp Edwards  Massachusetts
R

RAWS Site: Blackbird  Delaware
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txDBLB
RAWS Site: Prime Hook  Delaware
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txDPRI
RAWS Site: Redden  Delaware
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txDRED
RAWS Site: Big Sandy  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txKBSN
RAWS Site: Cherry Tree Knob  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txKCHE
RAWS Site: Jackson Co. AP  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txKJAC
RAWS Site: Triangle Mtn  Kentucky
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txKTRI
RAWS Site: Antietam NB  Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMANN
RAWS Site: Assateague Island     Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMASS
RAWS Site: Blackwater  Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?txMBLA
RAWS Site: Cedarville  Maryland
RAWS URL: https://raws.dri.edu/cgi-bin/wea_info.pl?

In [169]:
code_urls

{'https://raws.dri.edu/cgi-bin/wea_info.pl?caCCSH',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?txJRIN',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?wyWDYC',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?laFCEN',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?nmXGIL',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?laGBRA',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?ndOWIC',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?caCALC',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?coCHMT',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?ndTBAR',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?caCJAR',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?coCRAG',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?ndOBRB',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?ndSLAN',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?waWGRN',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?idINFK',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?akABIL',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?azAIRO',
 'https://raws.dri.edu/cgi-bin/wea_info.pl?txVKEA',
 'https://ra

In [137]:
def findstring(lst,string):
    '''
    find strings in a list, 
    '''
    my_list = list(lst)
    search_string = string
    found = []
    if type(my_list[0]) is dict:
        for nest in my_list:
            try:
                for value in nest.values():
                    if any(search_string in str(val) for val in value.values()):
                        found.append(nest)
            except:   
                for key, value in nest.items():
                    if search_string in value:
                        found.append(nest)     
                    else:
                        pass

        print('String found in the dictionary')
        return found        
            
    else:
        for item in my_list:
            if search_string in item:
                print("String found in the list.")
                found.append(item)
                break
        else:
            print("String not found in the list.")
    return found         
findstring(code_urls, 'nvNLIT')


String found in the list.


['https://raws.dri.edu/cgi-bin/wea_info.pl?nvNLIT']

In [34]:
list(code_urls)[0]

'https://raws.dri.edu/cgi-bin/wea_info.pl?ndNSCK'

In [86]:
def find_code2(url):
    '''
    Use to find 4-digit code with nessid info
    '''
    site_id = {}
    site_ids = {}
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        raws_links = soup.find_all('b')

        for tag in raws_links:
            td_tag = tag.find_all_next('td', limit=1)
            
            if td_tag:
                print(tag.get_text(), td_tag[0].get_text())
                site_id[tag.get_text()] = " ".join(str(td_tag[0]).split()[1:-1])

    site_ids[url[-4:]] = site_id         
    return site_ids

# Process URLs in parallel
nessid2 = Parallel(n_jobs=-1, verbose=1)(
delayed(find_code2)(url) for url in list(code_urls)) 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-1)]: Done 2868 out of 2868 | elapsed: 33.6min finished


In [84]:
'https://raws.dri.edu/cgi-bin/wea_info.pl?azADEE'[-4:]

'ADEE'

In [102]:
len(nessid2)

2868

In [103]:
nessid2

[{'NSCK': {'Location ': 'Sand Creek North Dakota',
   'Latitude ': '46° 32\' 44"',
   'NESS ID ': '32553302',
   'Longitude ': '103° 31\' 06"',
   'NWS ID ': '323804',
   'Elevation ': '2700 ft.',
   'Agency ': 'BLM'}},
 {'WSND': {'Location ': 'Sand Island Wisconsin',
   'Latitude ': '47° 00\' 11"',
   'NESS ID ': '3275B4F8',
   'Longitude ': '90° 56\' 11"',
   'NWS ID ': '',
   'Elevation ': '615 ft.',
   'Agency ': 'NPS'}},
 {'XBEW': {'Location ': 'Bear Wallow New Mexico',
   'Latitude ': '33° 27\' 18"',
   'NESS ID ': '326C15C2',
   'Longitude ': '108° 39\' 54"',
   'NWS ID ': '292010',
   'Elevation ': '9953 ft.',
   'Agency ': 'USFS'}},
 {'CLOS': {'Location ': 'Los Pinos Creek Colorado',
   'Latitude ': '38° 12\' 48"',
   'Longitude ': '106° 45\' 27"',
   'Elevation ': '9600 ft.'}},
 {'HLUA': {'Location ': 'Lua Makika Hawaii',
   'Latitude ': '20° 33\' 45"',
   'NESS ID ': '327906EA',
   'Longitude ': '156° 33\' 47"',
   'NWS ID ': '',
   'Elevation ': '1100 ft.',
   'Agency ': 'S

In [187]:
# Flatten the nested structure
flattened_data = {list(d.keys())[0]: list(d.values())[0] for d in nessid2}

ness_df = pd.DataFrame(flattened_data).T

# Save to nessid.csv
# ness_df.to_csv('nessid.csv')

In [90]:
len(nessid2)

2868

In [11]:
import pandas as pd
df_nes = pd.DataFrame(nessid2)
df_nes

,Location,Latitude,NESS ID,Longitude,NWS ID,Elevation,Agency
0,Sand Creek North Dakota,"46° 32' 44""",32553302,"103° 31' 06""",323804,2700 ft.,BLM
1,Sand Island Wisconsin,"47° 00' 11""",3275B4F8,"90° 56' 11""",,615 ft.,NPS
2,Bear Wallow New Mexico,"33° 27' 18""",326C15C2,"108° 39' 54""",292010,9953 ft.,USFS
3,Los Pinos Creek Colorado,"38° 12' 48""",NaN,"106° 45' 27""",NaN,9600 ft.,NaN
4,Lua Makika Hawaii,"20° 33' 45""",327906EA,"156° 33' 47""",,1100 ft.,S&amp;PF
...,...,...,...,...,...,...,...
2863,McKittrick Texas,"31° 58' 39""",3237015E,"104° 45' 06""",,5004 ft.,NPS
2864,Brasstown #1 Georgia,"34° 48' 10""",327357C4,"83° 42' 36""",090701,3280 ft.,USFS
2865,Williams Lookout North Dakota,"46° 21' 43""",NaN,"102° 03' 43""",NaN,2375 ft.,NaN
2866,Kane Springs Nevada,"37° 15' 02""",325CC5DA,"114° 42' 27""",261604,4590 ft.,BLM


# -----------------------------------------------------------------------------------------

# 6. Random Extra EDA

In [648]:
results

[{'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txPALL': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txVFDE': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCNEW': 2023},
 None,
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCOPA': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOKE2': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOPEB': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNGAL': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?hiHKII': 2023},
 None,
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?laNJES': 2023},
 None,
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOCIN': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?ndITWR': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?coCLOO': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?azANAV': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCHOR': 2023},
 {'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?laLEVA': 2023},
 {'https://raws.dri.edu/cgi-bin/we

In [623]:
check_urls

{'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txPALL',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?txVFDE',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCNEW',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNJWF',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCOPA',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOKE2',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOPEB',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNGAL',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?hiHKII',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCWHI',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?laNJES',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNSMI',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?orOCIN',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?ndITWR',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?coCLOO',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?azANAV',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?caCHOR',
 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?laLEVA',
 'https://

In [580]:
check_url[11]

'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?nvNSMI'

In [565]:
form_v = form.find(name='form', text='Latest')
form_v

In [369]:
# Same as above but thee raws_links process is slightly different
# Iterate over each state
raws_site = {} 
site_list = []
ss_list = set()
for region, states in stationsMenu.items():
    print(f"{'='*100}")
    print(f"\n\033[91mRegion: {region}\033[0m\n")
    print(f"{'-'*100}")
    for state, url in states.items():
        full_url = f"https://raws.dri.edu/{url}"
        print(f"State: {state}")
        print(f"URL: {full_url}")

        # Retrieve HTML content of the website
        response = requests.get(full_url)
        html_content = response.text

        # BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the RAWS sites
        raws_links = soup.find_all('a', {'href': re.compile(r'.+')})

        # Iterate over the RAWS sites, and create url for each one
        for raws_link in raws_links:
            if raws_link['href'][-4:].isupper():
                raws_site[raws_link.text.strip()] = raws_link['href'][-4:]
                
                print(f"RAWS Name, State: {raws_link.text.strip()}")
                print(f"RAWS Site: {raws_link['href'][-4:]}\n")
                
                site_list.append(raws_link['href'][-4:])
                ss_list.add(raws_link['href'][-4:])
                

print(len(raws_site)) 

Region: Western U.S.
State: Alaska
URL: https://raws.dri.edu/aklst.html
RAWS Name, State: 100 Mile Creek        Alaska
RAWS Site: A100

RAWS Name, State: Alcan Hwy MI-1244       Alaska
RAWS Site: AALC

RAWS Name, State: Angel Creek  Alaska
RAWS Site: AANG

RAWS Name, State: Asik  Alaska
RAWS Site: AASI

RAWS Name, State: Berners Bay  Alaska
RAWS Site: ABBY

RAWS Name, State: Black Cape  Alaska
RAWS Site: ABCA

RAWS Name, State: Birch Creek  Alaska
RAWS Site: ABCK

RAWS Name, State: Bear Mtn.         Alaska
RAWS Site: ABEA

RAWS Name, State: Ben Creek              Alaska
RAWS Site: ABEN

RAWS Name, State: Bering Glacier  Alaska
RAWS Site: ABER

RAWS Name, State: Beaver (WBQ)        Alaska
RAWS Site: ABEV

RAWS Name, State: Big Lake  Alaska
RAWS Site: ABIL

RAWS Name, State: Black Lake  Alaska
RAWS Site: ABLK

RAWS Name, State: Blair Lakes  Alaska
RAWS Site: ABLL

RAWS Name, State: Bentalit  Alaska
RAWS Site: ABNT

RAWS Name, State: Bolio  Alaska
RAWS Site: ABOL

RAWS Name, State: Booth 

RAWS Name, State: Aqua Canyon - Bryce Canyon NP.
RAWS Site: AQUA

RAWS Name, State: Assay - Hatch 10SW
RAWS Site: ASSA

RAWS Name, State: Badger Spring - Ivins 14W  Utah
RAWS Site: BADG

RAWS Name, State: Barney Reservoir - Escalante 15SE
RAWS Site: BARN

RAWS Name, State: Beaver Dam  Utah
RAWS Site: BEAV

RAWS Name, State: Bryce Canyon  Utah
RAWS Site: BRYC

RAWS Name, State: Enterprise  Utah
RAWS Site: ENTR

RAWS Name, State: Kane Gulch - Blanding 23WSW
RAWS Site: KANE

RAWS Name, State: Lava Point             Utah
RAWS Site: LAVA

RAWS Name, State: Telegraph Flat - Kanab 17E  Utah
RAWS Site: TELE

RAWS Name, State: Buckskin  Utah
RAWS Site: UBCS

RAWS Name, State: Between the Creeks  Utah
RAWS Site: UBTC

RAWS Name, State: Deer Springs Point  Utah
RAWS Site: UDSP

RAWS Name, State: Diamond Valley (BAER #12)  Utah
RAWS Site: UDVY

RAWS Name, State: Four Mile Bench  Utah
RAWS Site: UFMB

RAWS Name, State: KFO Kanab  Utah
RAWS Site: UKFO

RAWS Name, State: Lake  Utah
RAWS Site: ULAK

R

RAWS Name, State: Barrel Springs      Nevada
RAWS Site: NBAR

RAWS Name, State: Bluewing Mountain   Nevada
RAWS Site: NBLU

RAWS Name, State: Brawley Peaks       Nevada
RAWS Site: NBRA

RAWS Name, State: Black Rock Playa  Nevada
RAWS Site: NBRP

RAWS Name, State: Buffalo Creek       Nevada
RAWS Site: NBUF

RAWS Name, State: Catnip Mountain     Nevada
RAWS Site: NCAT

RAWS Name, State: Cherry Valley  Nevada
RAWS Site: NCHV

RAWS Name, State: Coyote Canyon       Nevada
RAWS Site: NCYC

RAWS Name, State: Dead Camel Mountain Nevada
RAWS Site: NDEA

RAWS Name, State: Double H  Nevada
RAWS Site: NDOU

RAWS Name, State: Dry Canyon          Nevada
RAWS Site: NDRY

RAWS Name, State: Desert Springs       Nevada
RAWS Site: NDSS

RAWS Name, State: Fish Springs        Nevada
RAWS Site: NFIS

RAWS Name, State: Fox Mountain        Nevada
RAWS Site: NFOX

RAWS Name, State: Galena  Nevada
RAWS Site: NGAL

RAWS Name, State: Incline Village (RAWS)  Nevada
RAWS Site: NINC

RAWS Name, State: Juniper Spring

RAWS Name, State: Ash Meadows  Nevada
RAWS Site: NASH

RAWS Name, State: Austin  Nevada
RAWS Site: NAUS

RAWS Name, State: Buddy Adams  Nevada
RAWS Site: NBDY

RAWS Name, State: Brawley Peaks       Nevada
RAWS Site: NBRA

RAWS Name, State: Cherry Valley  Nevada
RAWS Site: NCHV

RAWS Name, State: Coils Creek         Nevada
RAWS Site: NCOI

RAWS Name, State: Combs Canyon        Nevada
RAWS Site: NCOM

RAWS Name, State: Dead Camel Mountain Nevada
RAWS Site: NDEA

RAWS Name, State: Desatoya Mountain   Nevada
RAWS Site: NDES

RAWS Name, State: Desert Springs       Nevada
RAWS Site: NDSS

RAWS Name, State: Fish Springs        Nevada
RAWS Site: NFIS

RAWS Name, State: Flat Spring  Nevada
RAWS Site: NFLA

RAWS Name, State: Galena  Nevada
RAWS Site: NGAL

RAWS Name, State: Incline Village (RAWS)  Nevada
RAWS Site: NINC

RAWS Name, State: Knox  Nevada
RAWS Site: NKNX

RAWS Name, State: Little Valley  Nevada
RAWS Site: NLIT

RAWS Name, State: Mt. Como  Nevada
RAWS Site: NMCO

RAWS Name, State: Or

RAWS Name, State: Ash Meadows  Nevada
RAWS Site: NASH

RAWS Name, State: Big Bend            Nevada
RAWS Site: NBIG

RAWS Name, State: Christmas Tree Pass Nevada
RAWS Site: NCHR

RAWS Name, State: Desert NWR  Nevada
RAWS Site: NDNW

RAWS Name, State: Kyle Canyon  Nevada
RAWS Site: NKYL

RAWS Name, State: Mountain Springs  Nevada
RAWS Site: NMOU

RAWS Name, State: Pahrump             Nevada
RAWS Site: NPAH

RAWS Name, State: Red Rock            Nevada
RAWS Site: NRER

RAWS Name, State: Toquop Wash         Nevada
RAWS Site: NTOQ

RAWS Name, State: Wamp Springs  Nevada
RAWS Site: NWAM

RAWS Name, State: Yucca Gap  Nevada
RAWS Site: NYUC

RAWS Name, State: Ahakahv Preserve  Arizona
RAWS Site: AAHA

RAWS Name, State: Cibola  Arizona
RAWS Site: ACBL

RAWS Name, State: Fort Yuma  Arizona
RAWS Site: AFYM

RAWS Name, State: Havasu                   Arizona
RAWS Site: AHAV

RAWS Name, State: Mohave Mountain          Arizona
RAWS Site: AMOH

RAWS Name, State: Union Pass             Arizona
RAWS S

...

RAWS Name, State: Cabo Rojo  Puerto Rico
RAWS Site: PCAB

RAWS Name, State: Cotton Valley  St. Croix
RAWS Site: PCOT

RAWS Name, State: Camp Santiago  Puerto Rico
RAWS Site: PCPS

RAWS Name, State: Guanica  Puerto Rico
RAWS Site: PGNC

RAWS Name, State: Marecao  Puerto Rico
RAWS Site: PMAR

RAWS Name, State: Vieques  Puerto Rico
RAWS Site: PVIE

Region: Northeastern U.S.
State: Connecticut
URL: https://raws.dri.edu/ct_ma_rilst.html
RAWS Name, State: Ancora Hospital  New Jersey
RAWS Site: JANC

RAWS Name, State: Blue Mountain Lakes  New Jersey
RAWS Site: JBLU

RAWS Name, State: Coyle Field  New Jersey
RAWS Site: JCOY

RAWS Name, State: Jackson  New Jersey
RAWS Site: JJAC

RAWS Name, State: New Middlesex County  New Jersey
RAWS Site: JMID

RAWS Name, State: Ringwood  New Jersey
RAWS Site: JRIN

RAWS Name, State: Teetertown  New Jersey
RAWS Site: JTEE

RAWS Name, State: Wallkill River NWR  New Jersey
RAWS Site: JWAL

RAWS Name, State: Blue Hills  Massachusetts
RAWS Site: MBLH

RAWS Name, 

RAWS Name, State: Blackbird  Delaware
RAWS Site: DBLB

RAWS Name, State: Prime Hook  Delaware
RAWS Site: DPRI

RAWS Name, State: Redden  Delaware
RAWS Site: DRED

RAWS Name, State: Ancora Hospital  New Jersey
RAWS Site: JANC

RAWS Name, State: Coyle Field  New Jersey
RAWS Site: JCOY

RAWS Name, State: Eb Forsythe  New Jersey
RAWS Site: JEBF

RAWS Name, State: Jackson  New Jersey
RAWS Site: JJAC

RAWS Name, State: New Middlesex County  New Jersey
RAWS Site: JMID

RAWS Name, State: Teetertown  New Jersey
RAWS Site: JTEE

RAWS Name, State: Woodbine  New Jersey
RAWS Site: JWOO

RAWS Name, State: Antietam NB  Maryland
RAWS Site: MANN

RAWS Name, State: Assateague Island     Maryland
RAWS Site: MASS

RAWS Name, State: Blackwater  Maryland
RAWS Site: MBLA

RAWS Name, State: Cedarville  Maryland
RAWS Site: MCDV

RAWS Name, State: Catoctin Mountain  Maryland
RAWS Site: MCTM

RAWS Name, State: Green Ridge  Maryland
RAWS Site: MGRR

RAWS Name, State: Powellville  Maryland
RAWS Site: MPOW

RAWS Na

RAWS Name, State: Blackbird  Delaware
RAWS Site: DBLB

RAWS Name, State: Prime Hook  Delaware
RAWS Site: DPRI

RAWS Name, State: Redden  Delaware
RAWS Site: DRED

RAWS Name, State: Ancora Hospital  New Jersey
RAWS Site: JANC

RAWS Name, State: Blue Mountain Lakes  New Jersey
RAWS Site: JBLU

RAWS Name, State: Coyle Field  New Jersey
RAWS Site: JCOY

RAWS Name, State: Eb Forsythe  New Jersey
RAWS Site: JEBF

RAWS Name, State: Jackson  New Jersey
RAWS Site: JJAC

RAWS Name, State: New Middlesex County  New Jersey
RAWS Site: JMID

RAWS Name, State: Ringwood  New Jersey
RAWS Site: JRIN

RAWS Name, State: Teetertown  New Jersey
RAWS Site: JTEE

RAWS Name, State: Wallkill River NWR  New Jersey
RAWS Site: JWAL

RAWS Name, State: Woodbine  New Jersey
RAWS Site: JWOO

RAWS Name, State: Antietam NB  Maryland
RAWS Site: MANN

RAWS Name, State: Cedarville  Maryland
RAWS Site: MCDV

RAWS Name, State: Catoctin Mountain  Maryland
RAWS Site: MCTM

RAWS Name, State: Green Ridge  Maryland
RAWS Site: MGR

RAWS Name, State: Blackbird  Delaware
RAWS Site: DBLB

RAWS Name, State: Prime Hook  Delaware
RAWS Site: DPRI

RAWS Name, State: Redden  Delaware
RAWS Site: DRED

RAWS Name, State: Ancora Hospital  New Jersey
RAWS Site: JANC

RAWS Name, State: Blue Mountain Lakes  New Jersey
RAWS Site: JBLU

RAWS Name, State: Coyle Field  New Jersey
RAWS Site: JCOY

RAWS Name, State: Eb Forsythe  New Jersey
RAWS Site: JEBF

RAWS Name, State: Jackson  New Jersey
RAWS Site: JJAC

RAWS Name, State: New Middlesex County  New Jersey
RAWS Site: JMID

RAWS Name, State: Ringwood  New Jersey
RAWS Site: JRIN

RAWS Name, State: Teetertown  New Jersey
RAWS Site: JTEE

RAWS Name, State: Wallkill River NWR  New Jersey
RAWS Site: JWAL

RAWS Name, State: Woodbine  New Jersey
RAWS Site: JWOO

RAWS Name, State: Antietam NB  Maryland
RAWS Site: MANN

RAWS Name, State: Cedarville  Maryland
RAWS Site: MCDV

RAWS Name, State: Catoctin Mountain  Maryland
RAWS Site: MCTM

RAWS Name, State: Green Ridge  Maryland
RAWS Site: MGR

RAWS Name, State: Blackbird  Delaware
RAWS Site: DBLB

RAWS Name, State: Prime Hook  Delaware
RAWS Site: DPRI

RAWS Name, State: Redden  Delaware
RAWS Site: DRED

RAWS Name, State: Big Sandy  Kentucky
RAWS Site: KBSN

RAWS Name, State: Cherry Tree Knob  Kentucky
RAWS Site: KCHE

RAWS Name, State: Jackson Co. AP  Kentucky
RAWS Site: KJAC

RAWS Name, State: Triangle Mtn  Kentucky
RAWS Site: KTRI

RAWS Name, State: Antietam NB  Maryland
RAWS Site: MANN

RAWS Name, State: Assateague Island     Maryland
RAWS Site: MASS

RAWS Name, State: Blackwater  Maryland
RAWS Site: MBLA

RAWS Name, State: Cedarville  Maryland
RAWS Site: MCDV

RAWS Name, State: Catoctin Mountain  Maryland
RAWS Site: MCTM

RAWS Name, State: Green Ridge  Maryland
RAWS Site: MGRR

RAWS Name, State: Powellville  Maryland
RAWS Site: MPOW

RAWS Name, State: Patuxent  Maryland
RAWS Site: MPTX

RAWS Name, State: Susquehanna  Maryland
RAWS Site: MSUS

RAWS Name, State: Tuckahoe  Maryland
RAWS Site: MTUC

RAWS Name, State: Blue Ro

In [368]:
raws_site

{'100 Mile Creek        Alaska': 'A100',
 'Alcan Hwy MI-1244       Alaska': 'AALC',
 'Angel Creek  Alaska': 'AANG',
 'Asik  Alaska': 'AASI',
 'Berners Bay  Alaska': 'ABBY',
 'Black Cape  Alaska': 'ABCA',
 'Birch Creek  Alaska': 'ABCK',
 'Bear Mtn.         Alaska': 'ABEA',
 'Ben Creek              Alaska': 'ABEN',
 'Bering Glacier  Alaska': 'ABER',
 'Beaver (WBQ)        Alaska': 'ABEV',
 'Big Lake  Alaska': 'ABIL',
 'Black Lake  Alaska': 'ABLK',
 'Blair Lakes  Alaska': 'ABLL',
 'Bentalit  Alaska': 'ABNT',
 'Bolio  Alaska': 'ABOL',
 'Booth Lake  Alaska': 'ABOO',
 'Box River  Alaska': 'ABOX',
 'Brigadier  Alaska': 'ABRI',
 'Broadview              Alaska': 'ABRO',
 'Buckland Uplands  Alaska': 'ABUU',
 'Cape Kiavak  Alaska': 'ACAK',
 'Campbell Creek  Alaska': 'ACAM',
 'Caribou Peak          Alaska': 'ACCR',
 'Chatanika              Alaska': 'ACHA',
 'Chicken Creek  Alaska': 'ACHC',
 'Chief Cove  Alaska': 'ACHF',
 'Chigmit Mtns  Alaska': 'ACHG',
 'Chisana                Alaska': 'ACHI',
 'Ch

In [479]:
raws_urls

{'100 Mile Creek        Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akA100',
 'Alcan Hwy MI-1244       Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAALC',
 'Angel Creek  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAANG',
 'Asik  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAASI',
 'Berners Bay  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABBY',
 'Black Cape  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABCA',
 'Birch Creek  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABCK',
 'Bear Mtn.         Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABEA',
 'Ben Creek              Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABEN',
 'Bering Glacier  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABER',
 'Beaver (WBQ)        Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABEV',
 'Big Lake  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABIL',
 'Black Lake  Alaska': 'https:

# 7. EDA Part 2

In [476]:
# Convert lists back to sets
set1 = set(ss_list)
set2 = set(s_list)

# Find the extra values
set2_extra = set2 - set1
set1_extra = set1 - set2


print(f"set2:{list(set2_extra)}\nset1:{list(set1_extra)}")

set2:[]
set1:['MWHM', 'CLVQ', 'HPHU', 'SSFS']


# Whats going on
 - There a 4 codes that share the same key(location name) with others
 - For some of these it because one statation is the older one and the other is the newer one

In [471]:
# for s in s_list:
'CLVQ' in s_list

False

In [478]:
raws_urls

{'100 Mile Creek        Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akA100',
 'Alcan Hwy MI-1244       Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAALC',
 'Angel Creek  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAANG',
 'Asik  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akAASI',
 'Berners Bay  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABBY',
 'Black Cape  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABCA',
 'Birch Creek  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABCK',
 'Bear Mtn.         Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABEA',
 'Ben Creek              Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABEN',
 'Bering Glacier  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABER',
 'Beaver (WBQ)        Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABEV',
 'Big Lake  Alaska': 'https://raws.dri.edu/cgi-bin/wea_dysimts.pl?akABIL',
 'Black Lake  Alaska': 'https:

In [377]:
# Creating a list of stations, and a dict so i can join based on name to main data
s_list = []
s_dict = {}
for raws_name, raws_url in raws_urls.items():
    # Pulling the last 4 letters
    raws_id = raws_url[-4:]
    s_list.append(raws_id)
    s_dict[raws_name] = raws_id
stations = list(set(s_list))

# stations_dict = set(s_dict.values())

In [378]:
len(s_dict)

2907

In [379]:
len(s_list)

2907

In [217]:
stations

['CCDA',
 'FWIL',
 'OSQU',
 'IHPK',
 'XSLK',
 'PDOL',
 'IKRI',
 'TARA',
 'CWAW',
 'WSAN',
 'NWMT',
 'CDUM',
 'TKYL',
 'ISHV',
 'MWEF',
 'ADEE',
 'ISHN',
 'XJAR',
 'ANOA',
 'MWEL',
 'ACAR',
 'XJEM',
 'IBUL',
 'OMUL',
 'ASAK',
 'SLON',
 'MCOY',
 'CBB4',
 'VKEA',
 'NCAS',
 'OBUR',
 'CATT',
 'CFIV',
 'AFAR',
 'IROU',
 'UBCS',
 'NASH',
 'GOKE',
 'TCON',
 'YBEL',
 'CTAN',
 'CLOO',
 'ASP2',
 'AGOO',
 'WSNI',
 'MTRY',
 'CSMR',
 'ARUC',
 'CGOR',
 'AKAI',
 'TGRA',
 'ILIN',
 'BIGI',
 'LLAC',
 'MMLK',
 'MGRB',
 'OEMI',
 'APAM',
 'AYAN',
 'MGOR',
 'MPHL',
 'JTEE',
 'INFK',
 'AKAL',
 'XBAR',
 'CLGA',
 'CSAD',
 'AKAV',
 'CLAH',
 'CORE',
 'NPAN',
 'ORID',
 'CCH2',
 'IMGP',
 'NNAC',
 'CMPK',
 'LCAM',
 'WWLC',
 'NBUX',
 'XTRU',
 'CMSA',
 'CGHP',
 'MBCB',
 'MMCF',
 'UBRB',
 'ANF2',
 'OBAD',
 'XMEL',
 'CCPT',
 'IRAF',
 'CWCN',
 'VASH',
 'WGLI',
 'CBRF',
 'OGAD',
 'TMCF',
 'OSWE',
 'ICBO',
 'CMAT',
 'NBKC',
 'ZIOC',
 'AMUS',
 'CSTO',
 'CWWC',
 'MEKH',
 'CRAN',
 'OCLV',
 'APAR',
 'CCAM',
 'WHUR',
 'HOAH',
 

In [213]:
# Checking lengths of list
len(stations)

2864

In [205]:
# checking lengths of dict, should match list
len(stations_dict)

2864

# 8. Pulling Final List of 4-digit Codes
- Pulled from f_list
- set equal to result
- result is used to make stations_list

In [724]:
def break_list_into_chunks(lst, chunk_size):
    '''
    Breaks input into chunks
    Used to create multiple lists to input into post request
    '''
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]

result = break_list_into_chunks(f_list, 250)

# 9. Creating Station_list

In [731]:
# convert the above result to dataframe
result = pd.DataFrame(result).T

In [733]:
# save to csv
# result.to_csv('station_list.csv', index=False)

# 10. Random Functions

In [169]:
def lengthfixer(nested_list):
    '''
    Iterates through a nested list and finds all indices that are different lengths
    The first indice is the standard length.
    '''

    differing_indices = []
    differing_length = None
  
    for index, sublist in enumerate(nested_list):
        if differing_length is None:
            differing_length = len(sublist)
        elif len(sublist) != differing_length:
            differing_indices.append(index)

    if differing_indices:
        for index in differing_indices:
            differing_sublist = nested_list[index]
            print(f"Index {index} has a different length: {len(differing_sublist)}")
    else:
        print("All indices have the same length")

lengthfixer(dataset)

All indices have the same length


In [154]:
def lengthfixer2(nested_list, num):
    '''
    Same purpose as lengthfixer but if the length of each indice is already known
    '''
    differing_indices = []

    for index, sublist in enumerate(nested_list):
        if len(sublist) != num:
            differing_indices.append(index)

    if differing_indices:
        for index in differing_indices:
            differing_sublist = nested_list[index]
            print(f"The sublist at index {index} has a different length: {len(differing_sublist)}")
    else:
        print("All sublists have the correct length.")
        
lengthfixer2(dataset, 15)

The sublist at index 3393 has a different length: 12
The sublist at index 3394 has a different length: 18
The sublist at index 3395 has a different length: 1
The sublist at index 3396 has a different length: 0
The sublist at index 3397 has a different length: 1
The sublist at index 3398 has a different length: 1
